# Account Details

## Packages & Parameters

In [1]:
from utils.packages import *
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [2]:
temp_file = "config/access_keys.yaml"
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)     
    
data = {}    

## Get Invest Details

In [3]:
def get_invest_details():
    global data
    data["access_token"] = config[data['account_name']]["token"]
    data["accountID"] = config[data['account_name']]["account_id"]
    data["api"] = API(access_token = data["access_token"])
    r = accounts.AccountDetails(data["accountID"])

    data["api"].request(r)
    data["account_data"] = r.response
    data["account_balance"] = float(data["account_data"]["account"]["balance"])

    data["invest_amount"] = data["account_balance"] / data["balance_invest_ratio"]
    data["order_amount"] = int(np.floor(data["invest_amount"] * data["leverage"]))


    print(f'account_balance: {data["account_balance"]} ')
    print(f'invest_amount: {data["invest_amount"]} ')
    print(f'order_amount: {data["order_amount"]} ')

In [4]:
data["balance_invest_ratio"] = 100
data["leverage"] = 50
data['account_name'] = 'oanda_demo_2'

get_invest_details()

account_balance: 100.0031 
invest_amount: 1.0000310000000001 
order_amount: 50 


## Get Net PL

In [24]:
def get_net_pl():
    global data
    
    start_date = datetime(year=data['start_date']['year'],
                          month=data['start_date']['month'],
                          day=data['start_date']['date'],
                          hour = data['start_date']['hour'],
                          minute=data['start_date']['min']) - dt.timedelta(hours=8)
    
    data["access_token"] = config[data['account_name']]["token"]
    data["accountID"] = config[data['account_name']]["account_id"]
    data["api"] = API(access_token = data["access_token"])
    
    
    request_obj = trans.TransactionList(accountID=data["accountID"], params={"pageSize": 1000})
    data["api"].request(request_obj)
    last_transaction = int(request_obj.response['lastTransactionID'])

    for trans_id in tqdm(reversed(range(1,last_transaction))):
        request_obj = trans.TransactionDetails(accountID=data["accountID"], transactionID = trans_id)
        data["api"].request(request_obj)
        trans_time_val = request_obj.response['transaction']['time']
        trans_date = dt.datetime.strptime(trans_time_val.split(".")[0], "%Y-%m-%dT%H:%M:%S")

        if trans_date >= start_date:
            first_trans_id = trans_id
        else:
            break

    params = {"from": first_trans_id, "to": last_transaction}
    request_obj = trans.TransactionIDRange(accountID=data["accountID"], params=params)
    data["api"].request(request_obj)
    all_transactions = request_obj.response   


    pl_list = []
    for t in all_transactions['transactions']:
        try:
            pl_val = float(t['pl'])
            if pl_val != 0.0:
                pl_list.append(pl_val)
        except:
            pass

    num_transactions = len(pl_list)
    net_pl = np.sum(pl_list)

    print('-------------------------------')
    print(f'account_name      : {data["account_name"]}')    
    print(f'start_ts          : {start_date}')
    print(f'first_trans_id    : {first_trans_id}')
    print(f'last_trans_id     : {last_transaction}')    
    print(f'num_transactions  : {num_transactions}')
    print(f'net_pl            : {net_pl}')
    print(f'pl_list           : {pl_list}')
    print('-------------------------------')
    return(all_transactions)

In [29]:
data['start_date'] = {'year':2021, 'month':3, 'date':9, 
                     'hour':15, 'min':0, 
                     }

In [30]:
account_list = ['oanda_demo_primary','oanda_demo_1', 'oanda_demo_2']

for acc_name in account_list:
    data['account_name'] = acc_name
    all_transactions = get_net_pl()

89it [00:23,  3.72it/s]


-------------------------------
account_name      : oanda_demo_primary
start_ts          : 2021-03-09 07:00:00
first_trans_id    : 4867
last_trans_id     : 4956
num_transactions  : 15
net_pl            : -0.0021000000000000003
pl_list           : [0.0006, -0.0004, 0.0006, -0.0004, -0.0004, 0.0008, -0.0005, -0.0005, -0.0005, -0.0004, -0.0004, -0.0004, 0.0004, -0.0003, -0.0003]
-------------------------------


65it [00:17,  3.66it/s]


-------------------------------
account_name      : oanda_demo_1
start_ts          : 2021-03-09 07:00:00
first_trans_id    : 5250
last_trans_id     : 5315
num_transactions  : 11
net_pl            : 0.0004000000000000001
pl_list           : [0.0006, 0.0007, -0.0008, 0.0007, 0.001, 0.0007, -0.0004, -0.0006, -0.0006, -0.0003, -0.0006]
-------------------------------


49it [00:13,  3.73it/s]


-------------------------------
account_name      : oanda_demo_2
start_ts          : 2021-03-09 07:00:00
first_trans_id    : 5
last_trans_id     : 54
num_transactions  : 7
net_pl            : 0.0031000000000000003
pl_list           : [0.0006, 0.0007, 0.0008, 0.0009, 0.0007, 0.0003, -0.0009]
-------------------------------


data['account_name'] = 'oanda_demo_2'
all_transactions = get_net_pl()

data['account_name'] = 'oanda_demo_1'
all_transactions = get_net_pl()

data['account_name'] = 'oanda_demo_primary'
all_transactions = get_net_pl()